# Timeseries data

In this example we calculate the data of a wind farm with 67 turbines in a time series containing 8000 uniform inflow states.

The required imports are:

In [1]:
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar

import foxes
import foxes.variables as FV
from foxes.utils.runners import DaskRunner

First, we create the `model book`, adding the turbine type model (see examples above):

In [2]:
mbook = foxes.ModelBook()
mbook.turbine_types["NREL5"] = foxes.models.turbine_types.PCtFile(
    "NREL-5MW-D126-H90.csv"
)

Next, we create the `states`. The `data_source` can be any csv-type file (or `pandas` readable equivalent), or a `pandas.DataFrame` object. If it is a file path, then it will first be searched in the file system, and if not found, in the static data. If it is also not found there, an error showing the available static data file names is displayed.

In this example the static data file `timeseries_8000.csv.gz` will be used, with content
```
Time,ws,wd,ti
2017-01-01 00:00:00,15.62,244.06,0.0504
2017-01-01 00:30:00,15.99,243.03,0.0514
2017-01-01 01:00:00,16.31,243.01,0.0522
2017-01-01 01:30:00,16.33,241.26,0.0523
...
```
Notice the column names, and how they appear in the `foxes.Timeseries` constructor:

In [3]:
states = foxes.input.states.Timeseries(
    data_source="timeseries_8000.csv.gz",
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "ws", FV.WD: "wd", FV.TI: "ti"},
    fixed_vars={FV.RHO: 1.225},
)

Likewise, we create the example wind farm with 67 turbines from static data. The file `test_farm_67.csv` has the following structure:
```
index,label,x,y
0,T0,101872.70,1004753.57
1,T1,103659.97,1002993.29
2,T2,100780.09,1000779.97
3,T3,100290.42,1004330.88
...
```
For more options, check the API section `foxes.input.farm_layout`. 

We consider two turbine models in this example: the wind turbine type `NREL5` from above, and the model `kTI_02` from the `model book`. This model adds the variable `k` for each state and turbine, calculated as `k = kTI * TI`, with constant `kTI = 0.2`. The parameter `k` will later be used by the wake model.

In [4]:
farm = foxes.WindFarm()
foxes.input.farm_layout.add_from_file(
    farm, "test_farm_67.csv", turbine_models=["kTI_02", "NREL5"], verbosity=0
)

Next, we create the `algorithm`, with further model selections. In particular, two wake models are invoked, the model `Bastankhah_linear` for wind speed deficits and the model `CrespoHernandez_quadratic` for turbulence intensity:

In [5]:
algo = foxes.algorithms.Downwind(
    mbook,
    farm,
    states=states,
    rotor_model="centre",
    wake_models=["Bastankhah_quadratic", "CrespoHernandez_max"],
    wake_frame="rotor_wd",
    partial_wakes_model="auto",
    chunks={FV.STATE: 1000},
    verbosity=0,
)

Also notice the `chunks` parameter, specifying that always 1000 states should be considered in vectorized form during calculations. The progress can be visualized using `dask`'s `ProgressBar`:

In [6]:
with ProgressBar():
    farm_results = algo.calc_farm()

fr = farm_results.to_dataframe()
print("\n", fr[[FV.WD, FV.AMB_REWS, FV.REWS, FV.AMB_P, FV.P]])

[                                        ] | 0% Completed | 171.12 us

[########################################] | 100% Completed | 101.82 ms

[                                        ] | 0% Completed | 121.63 us

[##                                      ] | 6% Completed | 103.66 ms

[##                                      ] | 6% Completed | 204.80 ms

...

[###################################     ] | 88% Completed | 13.37 s

[########################################] | 100% Completed | 13.47 s

[                                        ] | 0% Completed | 329.00 us

[########################################] | 100% Completed | 109.31 ms


                                  WD  AMB_REWS       REWS    AMB_P            P
state               turbine                                                   
2017-01-01 00:00:00 0        244.06     15.62  15.598951  5000.00  5000.000000
                    1        244.06     15.62  14.326038  5000.00  5000.000000
                    2        244.06     15.62  15.067607  5000.00  5000.000000
                    3        244.06     15.62  15.522240  5000.00  5000.000000
                    4        244.06     15.62  14.832921  5000.00  5000.000000
...                             ...       ...        ...      ...          ...
2017-06-16 15:30:00 62       299.19     11.70   9.215285  4868.75  2718.771927
                    63       299.19     11.70  11.435150  4868.75  4752.878044
                    64       299.19     11.70  11.700000  4868.75  4868.750000
                    65       299.19     11.70  11.607320  4868.75  4828.202586
                    66       299.19     11.70  10.

For the fun of it, we can also run this example in parallel, on a local cluster. Depending on the system and the problem size, this is not neccessarily faster than the above implicitely used default dask scheduler, and it comes with overhead. But for complex calculations it is extremely useful and can really save the day. Read the [docs](https://docs.dask.org/en/stable/deploying.html) for more details and parameters. The following invokes the default settings for the local cluster:

In [7]:
with DaskRunner(scheduler="distributed") as runner:
    farm_results = runner.run(algo.calc_farm)

fr = farm_results.to_dataframe()
print("\n", fr[[FV.WD, FV.AMB_REWS, FV.REWS, FV.AMB_P, FV.P]])

Launching dask cluster..


LocalCluster(c3d7df32, 'tcp://127.0.0.1:40427', workers=4, threads=8, memory=15.49 GiB)
Dashboard: http://127.0.0.1:8787/status





Shutting down dask cluster



                                  WD  AMB_REWS       REWS    AMB_P            P
state               turbine                                                   
2017-01-01 00:00:00 0        244.06     15.62  15.598951  5000.00  5000.000000
                    1        244.06     15.62  14.326038  5000.00  5000.000000
                    2        244.06     15.62  15.067607  5000.00  5000.000000
                    3        244.06     15.62  15.522240  5000.00  5000.000000
                    4        244.06     15.62  14.832921  5000.00  5000.000000
...                             ...       ...        ...      ...          ...
2017-06-16 15:30:00 62       299.19     11.70   9.215285  4868.75  2718.771927
                    63       299.19     11.70  11.435150  4868.75  4752.878044
                    64       299.19     11.70  11.700000  4868.75  4868.750000
                    65       299.19     11.70  11.607320  4868.75  4828.202586
                    66       299.19     11.70  10.

Notice the `Dashboard` link, which is only valid during runtime and in this case is a `localhost` address. The dashboard gives plenty of information of the progress during the run and is a very useful tool provided by dask.

Note that the `DaskRunner` includes an option for the above mentioned `ProgressBar` (cf. API), so the function that is called within `runner.run(...)` should not also try to invoke it as well.